# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [2]:
# Load the libraries as required.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, OneHotEncoder, PowerTransformer, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle
import shap
import warnings
warnings.filterwarnings('ignore')

# Set random state for reproducibility
RANDOM_STATE = 42

/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [4]:

X = fires_dt.drop('area', axis=1)
y = fires_dt['area']

numerical_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()


In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=None
)


Training set shape: (413, 12)
Test set shape: (104, 12)
Training target mean: 11.13
Test target mean: 19.66

Training target statistics:
Zeros in training: 196 (47.5%)
Non-zeros in training: 217 (52.5%)


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [30]:
# Preproc 1: Simple preprocessing with scaling and one-hot encoding
preproc1 = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

print("Preproc1 created successfully!")
print("Numerical features to be scaled:", numerical_features)
print("Categorical features to be one-hot encoded:", categorical_features)


Preproc1 created successfully!
Numerical features to be scaled: ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
Categorical features to be one-hot encoded: ['month', 'day']


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [31]:
# Preproc 2: Preprocessing with power transformation and scaling
from sklearn.pipeline import make_pipeline

# Create a pipeline that applies power transformation followed by robust scaling
power_pipeline = make_pipeline(
    PowerTransformer(method='yeo-johnson', standardize=False),
    RobustScaler()
)

preproc2 = ColumnTransformer(
    transformers=[
        ('num', power_pipeline, numerical_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

print("Preproc2 created successfully!")
print("Numerical features will undergo power transformation + robust scaling")
print("Categorical features to be one-hot encoded:", categorical_features)
print("Power transformation (Yeo-Johnson) will make skewed variables more normal-like")


Preproc2 created successfully!
Numerical features will undergo power transformation + robust scaling
Categorical features to be one-hot encoded: ['month', 'day']
Power transformation (Yeo-Johnson) will make skewed variables more normal-like


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [32]:
# Pipeline A = preproc1 + baseline (Ridge Regression)
pipeline_A = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', Ridge(random_state=RANDOM_STATE))
])

print("Pipeline A created: preproc1 + Ridge Regression")


Pipeline A created: preproc1 + Ridge Regression


In [33]:
# Pipeline B = preproc2 + baseline (Ridge Regression)
pipeline_B = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', Ridge(random_state=RANDOM_STATE))
])

print("Pipeline B created: preproc2 + Ridge Regression")




Pipeline B created: preproc2 + Ridge Regression


In [34]:
# Pipeline C = preproc1 + advanced model (Random Forest)
pipeline_C = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])

print("Pipeline C created: preproc1 + Random Forest Regressor")


Pipeline C created: preproc1 + Random Forest Regressor


In [35]:
# Pipeline D = preproc2 + advanced model (Random Forest)
pipeline_D = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])

print("Pipeline D created: preproc2 + Random Forest Regressor")

# Store all pipelines for easy access
pipelines = {
    'Pipeline_A': pipeline_A,
    'Pipeline_B': pipeline_B, 
    'Pipeline_C': pipeline_C,
    'Pipeline_D': pipeline_D
}

print(f"\nAll {len(pipelines)} pipelines created successfully!")


Pipeline D created: preproc2 + Random Forest Regressor

All 4 pipelines created successfully!


# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [44]:
# Hyperparameter tuning for Pipeline A (preproc1 + Ridge)
param_grid_A = {'regressor__alpha': [0.1, 1.0, 10.0, 100.0]}

grid_search_A = GridSearchCV(
    pipeline_A, 
    param_grid_A, 
    cv=5, 
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1)



In [45]:
# Hyperparameter tuning for Pipeline B (preproc2 + Ridge)
param_grid_B = {'regressor__alpha': [0.1, 1.0, 10.0, 100.0]}

grid_search_B = GridSearchCV(
    pipeline_B, 
    param_grid_B, 
    cv=5, 
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1)


In [46]:
# Hyperparameter tuning for Pipeline C (preproc1 + Random Forest)
param_grid_C = { 'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [5, 10, None]}

grid_search_C = GridSearchCV(
    pipeline_C, 
    param_grid_C, 
    cv=5, 
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1)


In [47]:
# Hyperparameter tuning for Pipeline D (preproc2 + Random Forest)
param_grid_D = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [5, 10, None]}

grid_search_D = GridSearchCV(
    pipeline_D, 
    param_grid_D, 
    cv=5, 
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1)


In [ ]:

grid_searches = {
    'Pipeline_A': grid_search_A,
    'Pipeline_B': grid_search_B,
    'Pipeline_C': grid_search_C,
    'Pipeline_D': grid_search_D
}

results = {}
for name, grid_search in grid_searches.items():
    print(f"Fitting {name}...")
    grid_search.fit(X_train, y_train)
    results[name] = {
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_,
        'best_estimator': grid_search.best_estimator_
    }
    print(f"{name} - Best CV Score (neg_MSE): {grid_search.best_score_:.4f}")
    print(f"{name} - Best Parameters: {grid_search.best_params_}\n")

print("Hyperparameter tuning completed for all pipelines!")


Fitting Pipeline_A...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Pipeline_A - Best CV Score (neg_MSE): -2095.0220
Pipeline_A - Best Parameters: {'regressor__alpha': 100.0}

Fitting Pipeline_B...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Pipeline_B - Best CV Score (neg_MSE): -2098.5730
Pipeline_B - Best Parameters: {'regressor__alpha': 100.0}

Fitting Pipeline_C...
Fitting 5 folds for each of 9 candidates, totalling 45 fits


/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packag

Pipeline_C - Best CV Score (neg_MSE): -2709.9715
Pipeline_C - Best Parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 50}

Fitting Pipeline_D...
Fitting 5 folds for each of 9 candidates, totalling 45 fits


/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packag

Pipeline_D - Best CV Score (neg_MSE): -2696.4242
Pipeline_D - Best Parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 50}

Hyperparameter tuning completed for all pipelines!


/opt/miniconda3/envs/dsi_p/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [43]:

test_results = {}
for name, result in results.items():
    best_model = result['best_estimator']
    
    y_pred = best_model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    test_results[name] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'CV_Score': result['best_score'],
        'best_params': result['best_params']
    }
    
    print(f"{name} Test Results:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R2 Score: {r2:.4f}")
    print(f"  Best Parameters: {result['best_params']}")
    print()

Pipeline_A Test Results:
  RMSE: 108.4377
  MAE: 24.1443
  R2 Score: 0.0025
  Best Parameters: {'regressor__alpha': 100.0}

Pipeline_B Test Results:
  RMSE: 108.3756
  MAE: 24.0966
  R2 Score: 0.0036
  Best Parameters: {'regressor__alpha': 100.0}

Pipeline_C Test Results:
  RMSE: 109.4042
  MAE: 26.5349
  R2 Score: -0.0154
  Best Parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 50}

Pipeline_D Test Results:
  RMSE: 110.4351
  MAE: 26.6906
  R2 Score: -0.0346
  Best Parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 50}



# Evaluate

+ Which model has the best performance?

pipeline a

# Export

+ Save the best performing model to a pickle file.

In [ ]:
# Export the best performing model (Pipeline_A) to a pickle file
import os

# Create models directory if it doesn't exist
os.makedirs('../../05_src/models', exist_ok=True)

# Get the best model (Pipeline_A)
best_pipeline_name = 'Pipeline_A'  # Based on our analysis above
best_model = results[best_pipeline_name]['best_estimator']

# Save the best model
model_filename = '../../05_src/models/best_forest_fire_model.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(best_model, f)

print(f"✅ Best model ({best_pipeline_name}) saved to: {model_filename}")

# Also save model metadata for future reference
model_info = {
    'pipeline_name': best_pipeline_name,
    'model': best_model,
    'test_metrics': test_results[best_pipeline_name],
    'cv_score': results[best_pipeline_name]['best_score'],
    'best_params': results[best_pipeline_name]['best_params'],
    'feature_names': list(X.columns),
    'numerical_features': numerical_features,
    'categorical_features': categorical_features,
    'preprocessing': 'RobustScaler + OneHotEncoder',
    'algorithm': 'Ridge Regression'
}

metadata_filename = '../../05_src/models/model_metadata.pkl'
with open(metadata_filename, 'wb') as f:
    pickle.dump(model_info, f)

print(f"✅ Model metadata saved to: {metadata_filename}")
print(f"\nModel Performance Summary:")
print(f"  - Algorithm: Ridge Regression")
print(f"  - Preprocessing: Simple scaling + one-hot encoding")
print(f"  - Best Parameters: {results[best_pipeline_name]['best_params']}")
print(f"  - Test RMSE: {test_results[best_pipeline_name]['RMSE']:.4f}")
print(f"  - Test R²: {test_results[best_pipeline_name]['R2']:.4f}")


In [ ]:
# SHAP Explanations for Pipeline_A (Best Model)

# Get the best model for SHAP analysis
best_model = results['Pipeline_A']['best_estimator']

# Transform the data using the preprocessing pipeline
X_train_transformed = best_model.named_steps['preprocessing'].transform(X_train)
X_test_transformed = best_model.named_steps['preprocessing'].transform(X_test)

# Get feature names after preprocessing
preprocessor = best_model.named_steps['preprocessing']
feature_names = []

# Get numerical feature names (same as original)
feature_names.extend(numerical_features)

# Get categorical feature names after one-hot encoding
cat_transformer = preprocessor.named_transformers_['cat']
cat_feature_names = cat_transformer.get_feature_names_out(categorical_features)
feature_names.extend(cat_feature_names)

print(f"Total features after preprocessing: {len(feature_names)}")
print(f"Feature names: {feature_names}")

# Create SHAP explainer for the Ridge regression model
explainer = shap.Explainer(best_model.named_steps['regressor'], X_train_transformed, feature_names=feature_names)

# Calculate SHAP values for test set (using a sample to speed up computation)
sample_size = min(50, len(X_test_transformed))  # Use up to 50 test samples
X_test_sample = X_test_transformed[:sample_size]
shap_values = explainer(X_test_sample)

print(f"\n📊 SHAP Analysis completed for {sample_size} test samples")

# 1. Local explanation for a specific observation
sample_idx = 0  # First test observation
sample_prediction = best_model.predict(X_test.iloc[[sample_idx]])[0]
actual_value = y_test.iloc[sample_idx]

print(f"\n🔍 LOCAL EXPLANATION (Test Sample {sample_idx}):")
print(f"  - Actual fire area: {actual_value:.2f} ha")
print(f"  - Predicted fire area: {sample_prediction:.2f} ha")
print(f"  - Prediction error: {abs(actual_value - sample_prediction):.2f} ha")

# Show top contributing features for this sample
sample_shap = shap_values[sample_idx]
feature_contributions = [(feature_names[i], sample_shap.values[i]) for i in range(len(feature_names))]
feature_contributions.sort(key=lambda x: abs(x[1]), reverse=True)

print(f"\n  Top 5 most important features for this prediction:")
for i, (feature, contribution) in enumerate(feature_contributions[:5]):
    direction = "increases" if contribution > 0 else "decreases"
    print(f"  {i+1}. {feature}: {contribution:.3f} ({direction} prediction)")

# 2. Global feature importance
print(f"\n🌐 GLOBAL FEATURE IMPORTANCE:")
print("  Features ranked by average absolute SHAP value across all test samples:")

# Calculate mean absolute SHAP values
mean_shap_values = np.abs(shap_values.values).mean(axis=0)
global_importance = [(feature_names[i], mean_shap_values[i]) for i in range(len(feature_names))]
global_importance.sort(key=lambda x: x[1], reverse=True)

print(f"\n  Most important features (top 8):")
for i, (feature, importance) in enumerate(global_importance[:8]):
    print(f"  {i+1}. {feature}: {importance:.4f}")

print(f"\n  Least important features (bottom 5):")
for i, (feature, importance) in enumerate(global_importance[-5:], len(global_importance)-4):
    print(f"  {i}. {feature}: {importance:.4f}")

# 3. Feature selection recommendations
print(f"\n💡 FEATURE SELECTION RECOMMENDATIONS:")

# Identify very low importance features
low_importance_threshold = 0.01
low_importance_features = [feat for feat, imp in global_importance if imp < low_importance_threshold]

if low_importance_features:
    print(f"\n  Features that could be considered for removal (importance < {low_importance_threshold}):")
    for feat in low_importance_features:
        print(f"  - {feat}")
    
    print(f"\n  🧪 How to test feature removal:")
    print(f"  1. Create a new pipeline excluding these {len(low_importance_features)} features")
    print(f"  2. Use cross-validation to compare RMSE with/without these features")
    print(f"  3. If RMSE doesn't significantly increase, remove them for model simplicity")
    print(f"  4. Consider statistical tests (e.g., permutation importance) for validation")
else:
    print(f"  All features show importance ≥ {low_importance_threshold}")
    print(f"  Consider removing features with importance < {global_importance[-1][1]:.4f} (lowest)")

print(f"\n  📈 Feature enhancement testing strategy:")
print(f"  1. Use recursive feature elimination (RFE) with the Ridge model")
print(f"  2. Compare nested cross-validation scores with different feature subsets")
print(f"  3. Apply statistical feature selection (f_regression, mutual_info_regression)")
print(f"  4. Test feature interactions (polynomial features) for top predictors")

# Summary statistics
print(f"\n📋 SHAP ANALYSIS SUMMARY:")
print(f"  - Model: Ridge Regression (alpha={results['Pipeline_A']['best_params']['regressor__alpha']})")
print(f"  - Total features: {len(feature_names)}")
print(f"  - Most important: {global_importance[0][0]} (SHAP value: {global_importance[0][1]:.4f})")
print(f"  - Least important: {global_importance[-1][0]} (SHAP value: {global_importance[-1][1]:.4f})")
print(f"  - Features with importance < 0.01: {len(low_importance_features)}")


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.